### Imports

In [1]:
% matplotlib inline

import sys
import os
os.chdir('../..')
sys.path.append('../..')

import matplotlib.cm as cm
import matplotlib.pyplot as plt
import scipy.io as sio
import skimage.io as io
from IPython.display import Image
from scipy.sparse import csr_matrix
from scipy.spatial.distance import pdist

from pathlib import Path

import evaluate_lv as evaluate

from project.utils import labeller_lv

# Perform Evaluation

How well do the models perform on your test data?

#### Where are the models located?
Which models do you want to evaluate? Please supply the parent directory of the model directories.

In [2]:
# Where are the models located?
models_dir = '/media/taopan/data/landmark/00_project-master/models/lv'

filename = Path(models_dir) / 'model.ckpt-4668'

# Where are the .tfrecords?
test_data = '/media/taopan/data/landmark/00_project-master/data/tfrecords/lv/test.tfrecords'

#### Are you ready?
Run the next cell to evaluate the model.
Evaluation metrics returned in a Python dictionary.

In [3]:
import project.hourglass.params as hgparams

params = {
    hgparams.N_FEATURES: 256,
    hgparams.N_HOURGLASS: 4,
    hgparams.N_RESIDUALS: 3,
}

In [4]:
def get_immediate_subdirectories(models_dir):
    return [name for name in os.listdir(models_dir)
            if os.path.isdir(os.path.join(models_dir, name))]


print('Will perform evaluation on the following models in `{}`'.format(models_dir))
print(get_immediate_subdirectories(models_dir))
print()


metrics_for_everybody = {}

for name in get_immediate_subdirectories(models_dir):
    model_dir = os.path.join(models_dir, name)
    metrics = evaluate.evaluate(model_dir, test_data, params)
    metrics_for_everybody[name] = metrics
    
for model_dir, metrics in metrics_for_everybody.items():
    print('==========={}========'.format(model_dir))
    for metric, value in metrics.items():
        print('{} = {}'.format(metric, value))

Will perform evaluation on the following models in `/media/taopan/data/landmark/00_project-master/models/lv`
['lv_4hg_lr1e-3_decay10']
()
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fe88c4e2190>, '_model_dir': '/media/taopan/data/landmark/00_project-master/models/lv/lv_4hg_lr1e-3_decay10', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_session_config': None, '_tf_random_seed': None, '_environment': 'local', '_num_worker_replicas': 0, '_task_id': 0, '_save_summary_steps': 100, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_evaluation_master': '', '_master': ''}
DEBUG:tensorflow:Setting feature info to {'image': TensorSignature(dtype=tf.float32, shape=TensorShape([Dimension(4), Dimension(256), Dimension(256), Di

## The results are in!

In [6]:
for model_dir, metrics in metrics_for_everybody.items():
    print('==={}========'.format(model_dir))
    for metric, value in metrics.items():
        print('{} = {} <type is `{}`>'.format(metric, value, type(value)))

===lv_4hg_lr1e-3_decay10========
normalised_mean_error/in = 38.0465011597 <type is `<type 'numpy.float32'>`>
global_step = 4668 <type is `<type 'numpy.int64'>`>
loss = 0.00212945230305 <type is `<type 'numpy.float32'>`>
normalised_mean_error/all = 45.4765396118 <type is `<type 'numpy.float32'>`>
pck/in = 0.0147058824077 <type is `<type 'numpy.float32'>`>
normalised_mean_error/out = 40.9815177917 <type is `<type 'numpy.float32'>`>
pck/out = 0.0 <type is `<type 'numpy.float32'>`>
pck/all = 0.0 <type is `<type 'numpy.float32'>`>


#### Save to .json

In [ ]:
# import json
# import numpy as np

# class MyEncoder(json.JSONEncoder):
#     def default(self, obj):
#         if isinstance(obj, np.integer):
#             return int(obj)
#         elif isinstance(obj, np.floating):
#             return float(obj)
#         elif isinstance(obj, np.ndarray):
#             return obj.tolist()
#         else:
#             return super(MyEncoder, self).default(obj)
        
# def write_json(filename, model_metrics):
#     print('Writing JSON results to {}'.format(filename))
#     with open(filename, 'w') as handle:
#       json.dump(metrics_for_everybody, handle, sort_keys=True, indent=4, cls=MyEncoder)
    
# if metrics_for_everybody:
#     write_json('{}.json'.format(filename), metrics_for_everybody)

#### Save to .csv

In [7]:
import pandas as pd

def write_csv(filename, model_metrics):
    print('Writing CSV results to {}'.format(filename))
    df = pd.DataFrame(metrics_for_everybody)
    df.to_csv(filename)
    
if metrics_for_everybody:
    
    print(os.getcwd())
    
    write_csv('{}.csv'.format(filename), metrics_for_everybody)

/media/taopan/data/landmark/00_project-master
Writing CSV results to /media/taopan/data/landmark/00_project-master/models/lv/model.ckpt-4668.csv


#### Restore from .csv

In [8]:
def csv_to_dict(filename):
    dic = pd.DataFrame.from_csv(filename).to_dict()
    return dic

dic = csv_to_dict('{}.csv'.format(filename))

for model_dir, metrics in dic.items():
    print('==========={}========'.format(model_dir))
    for metric, value in metrics.items():
        print('{} = {}'.format(metric, value))

===========lv_4hg_lr1e-3_decay10========
loss = 0.00212945230305
global_step = 4668.0
normalised_mean_error/out = 40.9815177917
normalised_mean_error/all = 45.4765396118
pck/in = 0.0147058824077
pck/out = 0.0
pck/all = 0.0
normalised_mean_error/in = 38.0465011597
